In [1]:
setwd("/scratch/cloudstor/Modules Paper/Sept 2022/October version/Scripts Anna January 2023/Figure 3G")

In [2]:
#install.packages("circlize", lib=".")

In [3]:
library(dplyr)
#?library


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [4]:
#install.packages("circlize", lib=".")

library(circlize, lib.loc=".")

circlize version 0.4.15
CRAN page: https://cran.r-project.org/package=circlize
Github page: https://github.com/jokergoo/circlize
Documentation: https://jokergoo.github.io/circlize_book/book/

If you use it in published research, please cite:
Gu, Z. circlize implements and enhances circular visualization
  in R. Bioinformatics 2014.

This message can be suppressed by:
  suppressPackageStartupMessages(library(circlize))




### First get the top 100 most highly connected genes in the LGG purple module

Load in data on WGCNA modules from the LGG cohort

In [5]:
load("Subnetworks_LGG_tumour.Rdata")

In [6]:
ls()

[1] "sub_networks"

#### get the Purple module

##### Then Set lower half of matrix to

In [7]:
purple_module <- sub_networks$tumour$LGG$purple

### C5orf62 is a top connected gene that should become SMIM3 to match the genome bed file
colnames(purple_module)[ grep("C5orf", colnames(purple_module)) ] <- "SMIM3"
rownames(purple_module)[ grep("C5orf", rownames(purple_module)) ] <- "SMIM3"

diag(purple_module) <- 0
purple_module[ lower.tri(purple_module) ] <- 0

### Get the top 100 most strongly connected pairs of genes in the networks

In [8]:
top.connections <- head(sort(as.vector(purple_module[ upper.tri(purple_module) ]), decreasing = T) ,100)

top100 <- which(purple_module >= top.connections[100], arr.ind = TRUE)

top100.pairs.purple <- cbind( rownames(top100), colnames(purple_module)[ top100[,2] ] )


### Find the top connections for EGFR

In [9]:
grep("EGFR", rownames(purple_module))

top50.EGFR.links <- head(sort(c(purple_module[75,],purple_module[,75]), decreasing = T), 50)

top50.EGFR.links <- cbind(names(top50.EGFR.links), rep("EGFR",50), top50.EGFR.links ) 

[1] 75

### Now get genome co-ordinates of genes in purple module:

In [10]:
hg19_bed <- read.table("Homo_sapiens.GRCh37.87.genes.bed", sep="\t")

In [11]:
purple_hg19_bed <- hg19_bed[ hg19_bed$V9 %in% colnames(purple_module) , c(1,4,5,9)]
purple_hg19_bed[,1] <- paste("chr",purple_hg19_bed[,1], sep="")
colnames(purple_hg19_bed) <- c("Chr","Start","End","Gene")

### Add phylostrata data for each gene

In [12]:
Bckgrd.phylo <- read.csv("Phylostrata_PNAS.txt")

In [13]:
purple_hg19_bed.wPhylo <- inner_join(x=purple_hg19_bed, y=Bckgrd.phylo, by=c("Gene"="GeneID"), keep=T)

### Functions for assigning genes as unicellular (UC) or multicellular (MC)

In [14]:
color.by.age <- function(phylostratum) { 
  
  if (phylostratum <= 0 )  { return("NA") }
  
  if( is.na(phylostratum) ) { return("gray"); }
  
  if( phylostratum <= 3 ) { return("red") }
  
  else { return("blue") }
}

In [15]:
### Give genes a value according to UC or MC status
value.by.UC.MC <- function(phylostratum) { 
  
  if (phylostratum <= 0 )  { return(NA) }
  
  if( is.na(phylostratum) ) { return(NA); }
  
  if( phylostratum <= 3 ) { return(0) }
  
  else { return(1) }
}

In [16]:
name.UC.MC <- function(phylostratum) { 
  
  if (phylostratum <= 0 )  { return("None") }
  
  if( is.na(phylostratum) ) { return("None"); }
  
  if( phylostratum <= 3 ) { return("Unicellular") }
  
  else { return("Multicellular") }
}

### Add UC/MC status to BED

In [17]:
purple_hg19_bed.wPhylo <- cbind( purple_hg19_bed.wPhylo, "Is.Multicellular"=vapply( purple_hg19_bed.wPhylo$Phylostrata, 
                                                                             FUN=value.by.UC.MC, FUN.VALUE=-1 ),
                                 "Type"=vapply( purple_hg19_bed.wPhylo$Phylostrata, FUN=name.UC.MC, FUN.VALUE="NA" )
)

### Prepare objects for CIRCOS plot

In [18]:
g.info <- filter(purple_hg19_bed.wPhylo,Gene %in% c("EGFR","ERBB2","CLIC1","MYL12A") )

#### Get the coordinates of the genes in first column of the top 100 strongest connected pairs

In [19]:
GeneA_locs <- apply( top100.pairs.purple, 1, FUN=function(ROW) {
  filter(purple_hg19_bed , Gene==ROW[1])[c(1:4)]
}  )

GeneA_locs_bed <- do.call(rbind, GeneA_locs)

### Get the coordinates of the genes in second column of the top 100 strongest connected pairs

In [20]:
GeneB_locs <- apply( top100.pairs.purple, 1, FUN=function(ROW) {
  filter(purple_hg19_bed, Gene==ROW[2])[c(1:4)]
}  )

GeneB_locs_bed <- do.call(rbind, GeneB_locs)

#### Reorder columns of BED file of top 100 pairs to fit with circlize's functions

In [21]:
bed=purple_hg19_bed.wPhylo[,c(1:3,8,7,9,4)]

### Make a BED with the strongest connections to EGFR

In [22]:
## Get locations of top 50 co-expression partners of EGFR
EGRF_partner_locs <- apply( top50.EGFR.links, 1, FUN=function(ROW) {
  filter(purple_hg19_bed, Gene==ROW[1])
}  )

EGRF_partner_locs_bed <- do.call(rbind, EGRF_partner_locs)

EGRF_locs <- apply( top50.EGFR.links, 1, FUN=function(ROW) {
  filter(purple_hg19_bed, Gene==ROW[2])
}  )

EGRF_locs_bed <- do.call(rbind, EGRF_locs)

### Set colors for UC & MC genes

In [23]:
age.colors <- c("red","blue")
names(age.colors)  <- c("Unicellular", "Multicellular")

## Now draw CIRCOS plot

In [26]:
## Initialize blank plot
pdf("Fig 3G.pdf", width=5, height=5)
circos.initializeWithIdeogram(plotType = NULL)

## Label genes of interest
circos.genomicLabels(g.info, labels.column = "GeneID", side = "outside",  col="orange", cex=0.7, padding=0,
                     labels_height=0.01 )

## Add chromsome tracks
circos.genomicIdeogram()

circos.track(track.index = get.current.track.index(), panel.fun = function(x, y) {
  circos.genomicAxis(h = "top")
})

## Show positions of genes as points, coloured by UC/MC status:
circos.genomicTrackPlotRegion(bed, numeric.column = 4, ylim = c(-0.5,1.5), ##track.height=5,
                              panel.fun = function(region, value, ...) {
                                circos.genomicPoints(region, value[[1]], pch = 16, cex = 1, col = age.colors[value[[3]]] )  #col="black" )
                                i = getI(...)
                                # circos.lines(CELL_META$cell.xlim, c(i, i), lty = 2, col = "#00000040")
                              })

circos.genomicLink(GeneA_locs_bed, GeneB_locs_bed, col = "grey80", lwd=2 )
circos.genomicLink(EGRF_locs_bed[1:48,], EGRF_partner_locs_bed, col = "orange", lwd=2 )

dev.off()

png 
  2